## Данный пример демонстрирует создание нейросети (НС) при помощи интерпретатора специального языка (названия пока нет)

Шаги создания модели:
1. Написать скрипт с архитектурой НС.
2. Создать экземпляр парсера Parser().
3. Создать модели pytorch из скрипта при помощи функции from_str().
4. Извлечь готовый модуль из результата для использования в вычислениях.

In [ ]:
import torch
from torchview import draw_graph
from  generator.interpreter import Interpreter
import generator.bricks as bricks
import sys
import time
import torch.nn.functional as F

In [ ]:
# Тест создания моделей из выражения строки
device = "cuda"

# Примеры
examples = dict(
    s1 = "output={ {@4->relu+@8->relu}^2 }%2->@16->softmax->linear(5);",
    s2 = "output={ {@16->relu+@16->sigmoid}^4 }%8->@16;",
    s3 = "output={ {@64->relu}^16 }%16;",
    s4 = "output = linear(5) -> softmax;",
    s5 = "output = { @5->@20 + @10->@20 + @20 } -> softmax;",
    s6 = """
        y = @64 + @64;          # y - параллельно соединены x и модуль из 64 нейронов
        z = @8 -> y;            # z - x последовательно соединен с y
        w = @8 ^ 4;             # w - 4 слоя по 8 нейронов последовательно соединены
        a = {@16 + @16} % 2;    # a - параллельно соединены два модуля x
        output = z -> w -> a -> {{@8 -> relu + @8 -> relu} ^ 2} % 2 -> @16 -> softmax;
    """
)
# создаем парсер
interpreter = Interpreter()

# Отмечаем время старта
start_time = time.time()

# создаем модели примеров
scripts = {name: interpreter.parse(s) for name, s in examples.items()}

# Отмечаем время окончания создания модели
end_time = time.time()

# Результат работы парсера - набор модулей models, в которых храняться модели.
# Чтобы использовать модель - мы можем обратиться к ней по имени соответствующей переменной из скрипта.
model = scripts["s6"].get("output").to(device)

print(f"Время обработки всех скриптов: {end_time-start_time}")

2024-11-12 21:23:05.548 | INFO     | generator.interpreter:parse:150 - Time to generate models: 0.014869213104248047
2024-11-12 21:23:05.611 | INFO     | generator.interpreter:parse:150 - Time to generate models: 0.06184244155883789
2024-11-12 21:23:05.861 | INFO     | generator.interpreter:parse:150 - Time to generate models: 0.24912810325622559
2024-11-12 21:23:05.863 | INFO     | generator.interpreter:parse:150 - Time to generate models: 0.001957416534423828
2024-11-12 21:23:05.868 | INFO     | generator.interpreter:parse:150 - Time to generate models: 0.004101991653442383
2024-11-12 21:23:05.892 | INFO     | generator.interpreter:parse:150 - Time to generate models: 0.022968769073486328


Время обработки всех скриптов: 0.3589482307434082


In [ ]:
interpreter.ast_root.serialize()

{'type': 'script',
 'modules': {'y': {'type': 'assignment',
   'name': 'y',
   'value': {'type': 'operation',
    'operator': 'plus',
    'left': {'type': 'module', 'module_type': 'Linear', 'params': [64]},
    'right': {'type': 'module', 'module_type': 'Linear', 'params': [64]}}},
  'z': {'type': 'assignment',
   'name': 'z',
   'value': {'type': 'operation',
    'operator': 'arrow',
    'left': {'type': 'module', 'module_type': 'Linear', 'params': [8]},
    'right': {'type': 'identifier', 'name': 'y'}}},
  'w': {'type': 'assignment',
   'name': 'w',
   'value': {'type': 'operation',
    'operator': 'power',
    'left': {'type': 'module', 'module_type': 'Linear', 'params': [8]},
    'right': {'type': 'number', 'value': 4}}},
  'a': {'type': 'assignment',
   'name': 'a',
   'value': {'type': 'operation',
    'operator': 'percent',
    'left': {'type': 'operation',
     'operator': 'plus',
     'left': {'type': 'module', 'module_type': 'Linear', 'params': [16]},
     'right': {'type': '

In [ ]:
# Подсчитаем размер модели
from generator.visualizers import model_input_shape, model_params_count


input_shape = (1,1)

# Тестируем работу модели на тестовом тензоре
x = torch.randn(input_shape).to(device)
y = model.to(device)(x)
print(f"Результат:\n{y}")

params_count = model_params_count(model)
print(f"Параметров: {params_count}")

print(f"Размерность входного тензора: {y.shape}")
print(f"Размерность выходного тензора: {y.shape}")

Результат:
tensor([[0.0714, 0.0824, 0.0764, 0.0475, 0.0510, 0.0474, 0.0485, 0.0540, 0.0670,
         0.0621, 0.0876, 0.0652, 0.0535, 0.0751, 0.0500, 0.0609]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
Параметров: 3456
Размерность входного тензора: torch.Size([1, 16])
Размерность выходного тензора: torch.Size([1, 16])


In [5]:
# Получим один из элементов модели по идентификатору подмодуля
print(f"Подсеть: \n{model.get_submodule('left')}")


Подсеть: 
Composition(
  (left): Composition(
    (left): Composition(
      (left): Composition(
        (left): Composition(
          (left): Linear(
            (linear): Linear(in_features=1, out_features=8, bias=True)
          )
          (right): {Linear(
            (linear): Linear(in_features=8, out_features=64, bias=True)
          )+Linear(
            (linear): Linear(in_features=8, out_features=64, bias=True)
          )}
        )
        (right): Multiplicator(
          (layers): ModuleList(
            (0): Linear(
              (linear): Linear(in_features=64, out_features=8, bias=True)
            )
            (1-3): 3 x Linear(
              (linear): Linear(in_features=8, out_features=8, bias=True)
            )
          )
        )
      )
      (right): Splitter(
        (models): ModuleList(
          (0-1): 2 x {Linear(
            (linear): Linear(in_features=8, out_features=16, bias=True)
          )+Linear(
            (linear): Linear(in_features=8, out

In [6]:
# Обратная конвертация модели в выражение
# Это выражение не является полноценным скриптом, т.к. не является выражением присвоения
model.expr_str(expand=True)


'{{{{{{linear(8)->{linear(64)+linear(64)}}->linear(8)^4}->{linear(16)+linear(16)}%2}->{{linear(8)->relu}+{linear(8)->relu}}^2%2}->linear(16)}->softmax}'

In [7]:
# Нарисуем диаграмму модели
from generator.visualizers import draw_model


input_size = (1, 10)
pic_path = './pic'
graph_name = 'test'
model_graph = draw_model(model, graph_name, pic_path)

print(f"Изображение сохранено в {pic_path}/{graph_name}.png")
model_graph.resize_graph(scale=3)
model_graph.visual_graph.view(graph_name)

Изображение сохранено в ./pic/test.png


'pic/test.pdf'

In [8]:
# Пример того, как можно использовать в вычислениях отдельный подмодуль модели
# Тестовый входной тензор для модели
input_size = (1, 8)
x = torch.randn(input_size).to(device)

# Так как в синтаксисе операции '+' и '->' являются бинарными, 
# построенные из таких выражений подмодули имеют имена left и right
# Извлечем элемент left.right из модели
chunk = model.left.right
print(chunk(x))

graph_name='left.right'
model_graph = draw_model(chunk, graph_name, pic_path)

# Более того, мы можем менять структуру, например, операцией decompose()
# Разделим на две части блок chunk
left, right = chunk.decompose()

# Создадим новый модуль как соединение left и  right
new_chunk = bricks.Connector(left, right).to("cuda")
print(new_chunk(x))

graph_name='decomose'
model_graph = draw_model(chunk, graph_name, pic_path)
print()
print(f"Результат разделения и склейки сохранен в {pic_path}/{graph_name}")

tensor([[ 0.2899,  0.4050,  0.2727, -0.3197, -0.2631, -0.5494, -0.2929, -0.3652,
          0.3767, -0.1967,  0.1993,  0.4289, -0.3217,  0.1560,  0.1202,  0.1042]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[ 2.4078e-02, -7.2190e-01,  1.1648e-01,  1.7959e-03, -8.2747e-02,
         -5.2526e-02, -2.6982e-01,  1.8857e-04,  4.1846e-01,  3.2105e-01,
          3.5988e-01, -1.4250e-01, -2.2909e-01, -6.6207e-01, -2.9037e-01,
         -6.1305e-01,  5.9944e-01, -1.7699e-01,  7.5098e-02,  3.9966e-01,
         -2.5356e-01,  2.1176e-01,  1.2429e-01,  2.1163e-01]], device='cuda:0',
       grad_fn=<CatBackward0>)

Результат разделения и склейки сохранен в ./pic/decomose


In [9]:
print(f"Было:\n{chunk}\n\n")
print(f"После chunk.decompose() и briks.Connector стало:\n{new_chunk}")